In [1]:
import requests
import re
from bs4 import BeautifulSoup
from newspaper import Article
import urllib.request

# Crawl data

In [2]:
req = ['https://www.thegioididong.com/dtdd?p=tu-4-7-trieu',
       'https://www.thegioididong.com/dtdd?p=tu-7-13-trieu',
       'https://www.thegioididong.com/dtdd?p=tren-13-trieu',
]
def crawlPhoneLinks(req):
    phoneLinks = set()
    for r in req:
        r1 = requests.get(r)
        coverpage = r1.content
        soup1 = BeautifulSoup(coverpage, 'html.parser')
        for a in soup1.find_all('a', href=True):
            if re.search('/dtdd/.*', a['href']):
                if a.find_all('div', class_='ratingresult'):
                    phoneLinks.add(a['href'])
        #print(len(phoneLinks))
    return phoneLinks
phoneLinks = crawlPhoneLinks(req)

def crawlRevLink(phoneLinks):
    reviewLinks=set()
    for link in phoneLinks:
        if link.count('https://www.thegioididong.com')==0:
            link = 'https://www.thegioididong.com'+link
        link = link +'/danh-gia?p='
        reviewLinks.add(link)
    #print(len(reviewLinks))
    return reviewLinks
reviewLinks = crawlRevLink(phoneLinks)

def crawlRev(reviewLinks):
    reviews = []
    for link in reviewLinks:
        i = 1
        while True:
            link = link+str(i)
            try:
                page = urllib.request.urlopen(link)
                soup = BeautifulSoup(page, 'html.parser')
                new_feeds = soup.find(
                    'ul', class_='ratingLst').find_all('li')

                article = Article(link)
                article.download()
                article.parse()
                soup = BeautifulSoup(article.html, 'html.parser')
                html = soup.findAll("div", {"class": "rc"})
                reviews = reviews + html

            except:
                break;
            i = i+1
    return reviews
reviews = crawlRev(reviewLinks)

In [3]:
comments = []
stars = []
labels = []
for li in reviews:
        rev = str(li).split('\n')
        if len(rev)>9:
            comment = rev[9].replace('<i>','').replace('</i>','')
            if comments.count(comment)==0:
                comments.append(comment)
                star = str(li).count('txtstar')
                stars.append(star)
                if star>=3:
                    labels.append(1)
                else: 
                    labels.append(0)
#print(len(comments))

# Removing Noise from the Data

In [4]:
import re
import pyvi
from pyvi import ViTokenizer #For split vietnamese words

In [5]:

REPLACE_NO_SPACE = re.compile("[.;:\',\"()\[\]]")#("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [line.lstrip() for line in reviews]
    reviews = [ViTokenizer.tokenize(rev) for rev in reviews]
    return reviews
comments_clean = preprocess_reviews(comments)

# Write to xlsx

In [6]:
import pandas as pd 
df = pd.DataFrame(list(zip(comments, stars, labels)),columns =[ 'comment', 'stars','label']) 

In [7]:
df[0:5]

,comment,stars,label
0,"Đã dùng được 1.5 tháng, thiết kế đẹp, camera s...",5,1
1,Sài gần 4 tháng tui thấy cũng ok lắm. M...,5,1
2,Có ai bị trưa 12h sạc pin nắng quá máy lên dc ...,4,1
3,Dây sạc của đt sạc ko vô . Chập chờn và ghi th...,5,1
4,Điện thoại rất là ok . Chơi game rất là mượt ....,5,1


In [15]:
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
dftrain = df.head(int(len(df)*0.8))

In [17]:
dftest = df.tail(len(df)-len(dftrain))

In [18]:
dftrain.to_excel('dataTrain.xlsx', sheet_name='dataTrain',startrow=0, startcol=0)
dftest.to_excel('dataTest.xlsx', sheet_name='dataTest',startrow=0, startcol=0)
df.to_excel('data.xlsx', sheet_name='data',startrow=0, startcol=0)